In [1]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import math
import time
import re

In [3]:
base="https://wolt.com/az/aze/baku/search?q="

In [5]:
path = r'C:\Users\Admin\Documents\python\jupyter\geckodriver.exe'
driver=webdriver.Firefox(executable_path=path)

In [6]:
driver.get(base)

In [7]:
try:
    pop_up_element=driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[3]/div/div[2]/div/div/div/div/div[2]/div[2]/button')
    pop_up_element.click()
except:
    print('no popup')

In [8]:
page=BeautifulSoup(driver.page_source)
rests_div=page.find_all(class_=re.compile('ListPage__listItem_'))

In [9]:
rests=[]
for rest in rests_div[:]:
    rest_info=dict()
    title_card=rest.find(class_=re.compile('VenueBanner__titleCard_'))
    rest_info['title']=title_card.find(class_=re.compile('VenueBanner__title_')).text
    rest_info['desc']=title_card.find(class_=re.compile('VenueBanner__description_')).text
    
    bottomRow=rest.find(class_=re.compile('VenueBanner__bottomRow_'))
    rest_info['exp']=bottomRow.find('span').text
    try:
        rating=bottomRow.find(class_=re.compile('RatingsTooltip__restaurantRating_')).text
        rest_info['rating']=float(rating.split('R')[0].split('·')[1])
    except:
        pass
    try:
        cuisine_tags=bottomRow.find(class_=re.compile('VenueBanner__description_')).text
        rest_info['cuisine_tags']=cuisine_tags.split('·')[1].strip()
    except:
        pass
    
    rest_info['img']=rest.find(class_=re.compile('ImageWithTransition__image_')).get('style').split('"')[1]
    rest_info['url']='https://wolt.com'+rest.a.get('href')
    rests.append(rest_info)

In [10]:
len(rests)

50

In [12]:
foods=[]
rest_pages=rests.copy()
for rest_info in rest_pages[:5]:
    driver.get(rest_info['url'])
    
    page=BeautifulSoup(driver.page_source)
    adr=page.find(class_=re.compile('VenueSideInfo__venueAddress_'))
    rest_info['cover_image']=page.find(class_=re.compile('VenueBackground__image_')).get('src')
    rest_info['adress']=adr.find_all('div')[0].text
    rest_info['post']=adr.find_all('div')[1].text
    rest_info['location']=adr.find('a').get('href')
    
    info_lines=page.find_all(class_=re.compile('VenueSide__infoLine_'))
    work_days=info_lines[0].div.text
    work_hours=info_lines[0].find_all('div')[1].text
    deliver_days=info_lines[1].div.text
    deliver_hours=info_lines[1].find_all('div')[1].text
    
    ######################## FOODS
    menu_sections=page.find(class_=re.compile('Menu__area_')).find_all('section')
    for section in menu_sections:
        food_info=dict()
        food_info['rest']=rest_info['title']
        food_info['food_type']=section.find(class_=re.compile('MenuCategoryHeader__category_')).text
        food_areas=section.find_all(class_=re.compile('MenuItem__contentArea_'))
        for food_content in food_areas:
            food_info['name']=food_content.find(class_=re.compile('MenuItem__name_')).text.split('×')[1].strip()
            food_info['desc']=food_content.find(class_=re.compile('MenuItem__description_')).text
            food_info['price']=food_content.find(class_=re.compile('MenuItem__price_')).text.replace('\xa0', '')
            try:
                food_info['img']=food_content.find(class_=re.compile('ImageWithTransition__image_')).get('style').split('"')[1]
            except:
                pass
        foods.append(food_info)
    ######################
    
    
    
    #########after click to get more info
    more_info=driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[2]/div/div[4]/div[2]/div[3]/button')
    more_info.click()
    
    page=BeautifulSoup(driver.page_source)
    rest_info['more_desc']=page.find(class_=re.compile('VenueSide__description_')).text
    contacts=page.find_all(class_=re.compile('VenueSide__linkInfoLine_'))
    rest_info['phone']=contacts[0].find('a').text
    rest_info['social_url']=contacts[1].find('a').text
    ###############
    sleep(1.5)


In [107]:
rest_frame=pd.DataFrame(rests)
rest_frame

,title,desc,exp,rating,cuisine_tags,img,url,cover_image,adress,post,location,more_desc,phone,social_url
0,BİR-İKİ Döner,Bakı şəhərinə dönərism konsepsiyasını yayan məkan,$,9.2,"dönər, şaurma, roll",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/biriki...,https://prod-wolt-venue-images-cdn.wolt.com/5d...,Yusif Mammadaliyev 27A,AZ1000 Baku,"https://maps.google.com/?q=40.37140467126463,4...",BİR-İKİ döner shop - Bakı şəhərinə dönərism ko...,+994124989149,https://www.facebook.com/BIRIKIDONER
1,Vapiano Uzeyir Hajibeyov,Italian Cuisine,$$,8.8,"pasta, pizza, salat",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/vapian...,https://prod-wolt-venue-images-cdn.wolt.com/5d...,"Uzeyir Hajibeyov 6/8, Sabail distr.",AZ1000 Baku,"https://maps.google.com/?q=40.371016410416594,...",People who meet in our relaxed setting share o...,+994125988118,https://www.vapiano.com/en/home/
2,Meatadore,Probably the best burger you ever had in town,$$,9.2,"qamburger, Çizburger, içkilər",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/meatadore,https://prod-wolt-venue-images-cdn.wolt.com/5d...,Zeynalabdin Taghiyev str. 6A,AZ1001 Baku,"https://maps.google.com/?q=40.369978312531934,...",Probably the best burger you ever had in town,+994507709966,https://www.facebook.com/meatadore/
3,Namlı Kebap,Türk mətbəxinin ən ləzzətli unvanı,$$$,8.8,"türk, kəbab, dönər",https://prod-wolt-venue-images-cdn.wolt.com/5f...,https://wolt.com/az/aze/baku/restaurant/namli,https://prod-wolt-venue-images-cdn.wolt.com/5f...,3 Mardanov brothers str.,AZ1000 Baku,"https://maps.google.com/?q=40.3726589894877,49...",Türk mətbəxinin ən ləzzətli unvanı,+994502064644,http://www.namlikebapbaku.az/
4,Shaurma №1 Nizami Küç.,1998-ci ildən,$$,9.2,"lahmacun, mucha pizza, pide",https://prod-wolt-venue-images-cdn.wolt.com/5e...,https://wolt.com/az/aze/baku/restaurant/shaurm...,https://prod-wolt-venue-images-cdn.wolt.com/5e...,Nizami Street 81,AZ1000 Baku,"https://maps.google.com/?q=40.37303592011353,4...",Azərbaycanın ilk “fast-food” brendi,+994552527605,https://www.facebook.com/pg/shaurma.az/about/?...
5,Central Baku,İncəlikləri ilə seçilən kafe və şiriniyyat evi,$$$,9.0,"desert, desertlər, balıq",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/centra...,https://prod-wolt-venue-images-cdn.wolt.com/5d...,75 Nizami str.,AZ1000 Baku,"https://maps.google.com/?q=40.37253536185212,4...",Central Baku - tarixi olan Nizami kuçəsində ye...,+994505024555,https://www.facebook.com/centralbakuaz/
6,SunSet Cafe,Klassik Amerika mətbəxi\n,$$$,9.0,"Amerika, meksikan",https://prod-wolt-venue-images-cdn.wolt.com/5e...,https://wolt.com/az/aze/baku/restaurant/sunset...,https://prod-wolt-venue-images-cdn.wolt.com/5e...,"Sabail dist, Hajı Zeynalabdin Taghıyev 8A",az1005 Baku,"https://maps.google.com/?q=40.3701173394329,49...",Klassik Amerika mətbəxi\n,+99412492 22 92,https://www.facebook.com/pg/SunsetCafeBaku/
7,PakDad Nizami,Ətin ən ləzzətli halı,$$,9.0,"kəbab, dönər, lahmacun",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/pakdad...,https://prod-wolt-venue-images-cdn.wolt.com/5d...,"87 Nizami St, Baku 1000",AZ1000 Baku,"https://maps.google.com/?q=40.37323151070397,4...","Hər şüfrədə dad, o da PakDad",+994512355231,https://pakdad.az
8,Tavuk Pilav,🍛❤️,$,8.4,"Eastern cuisine, acılı, türk",https://prod-wolt-venue-images-cdn.wolt.com/5f...,https://wolt.com/az/aze/baku/restaurant/tavuk-...,https://prod-wolt-venue-images-cdn.wolt.com/5f...,Haci Zeynalabdin Tagiyev street 16,AZ1005 Baku,"https://maps.google.com/?q=40.370644739605865,...",🍛❤️,+994503021206,https://www.instagram.com/pilov_plove/
9,Yoshi Sushi,"Sushi, əriştə və daha çox!",$$$,8.8,"suşi, sup, salat",https://prod-wolt-venue-images-cdn.wolt.com/5d...,https://wolt.com/az/aze/baku/restaurant/yoshis...,https://prod-wolt-venue-images-cdn.wolt.com/5d...,24B Haji Zey

In [108]:
food_frame=pd.DataFrame(foods)
food_frame

,rest,food_type,name,desc,price,img
0,BİR-İKİ Döner,KLASSIK 🌯,Çıtır Dürüm Dana Əti ilə,"Lavaş, dana əti, pomidor, fri kartofu, mayonez...","4,40₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
1,BİR-İKİ Döner,XÜSUSİ ✨,3 Lavaş,"Dönər, şaurma və falafel (ət və ya toyuq)","4,80₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
2,BİR-İKİ Döner,ƏLAVƏ 🍟,Kənd Kartofu,,"3,70₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
3,BİR-İKİ Döner,İÇKİLƏR 🥤,Sandora Alma® 250 ml,,"2,00₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
4,Vapiano Uzeyir Hajibeyov,Wolt Specials,Baltika Menu 🥤,Pizza Diavolo (donuz əti ilə) və salat Mista D...,"20,90₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
...,...,...,...,...,...,...
464,Namaste,ŞEFDƏN,Kholapury Qoyun,"Qovrulmuş hindistan ədvaları, kokos südü və xa...","21,50₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
465,Namaste,CURRY,Rogan Josh Qoyun Əti ilə,Qoyun əti rogan usulu ilə hazırlanmış ədvalı k...,"21,50₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
466,Namaste,VEGETARYAN MENU,Malai Kofta,Hind üsulu ilə hazırlanmış pendirlər qaymaq və...,"15,00₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...
467,Namaste,DÜYÜ,Tərəvəzli Qovrulmuş Düyü,Tərəvəzlərlə qovrulmuş düyü,"7,50₼",https://wolt-menu-images-cdn.wolt.com/menu-ima...


In [110]:
rest_frame.to_csv('wolt_rests.csv')
food_frame.to_csv('wolt_foods.csv')